In [1]:
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

RANDOM_STATE = 666666

In [2]:
# Import Data

df = pd.read_csv('bow_clean_final.csv', sep=',')
df = df.dropna(subset = ['Age of death'])

In [3]:
df.nunique()

Id                   23394
Name                 23283
Short description    15240
Gender                   6
Country                746
Occupation             894
Birth year             124
Death year              48
Manner of death         68
Age of death            91
Words                23323
dtype: int64

In [4]:
df.reset_index(drop=True, inplace=True)
df

Id              Name  \
0            Q42     Douglas Adams   
1           Q368  Augusto Pinochet   
2           Q407    Linda Lovelace   
3           Q409        Bob Marley   
4           Q410        Carl Sagan   
...          ...               ...   
23390  Q70706427     Hevrin Khalaf   
23391  Q70794825    Richard Tracey   
23392  Q71262658       Patrick Day   
23393  Q74642113   Baha Abu al-Ata   
23394  Q76161186     Sara Champion   

                                       Short description  Gender  \
0                            English writer and humorist    Male   
1               General, politician of Chile (1915-2006)    Male   
2      American pornographic actress, later anti-porn...  Female   
3               Jamaican singer, songwriter and musician    Male   
4      American astrophysicist, cosmologist, author a...    Male   
...                                                  ...     ...   
23390       Syrian Kurdish politician and civil engineer  Female   
23391                                   Australian judge    Male   
23392                                     American boxer    Male   
23393          Palestinian terrorist organization leader    Male   
23394                              British archaeologist  Female   

                        Country                       Occupation  Birth year  \
0                United Kingdom                           Artist      1952.0   
1                         Chile                       Politician      1915.0   
2      United States of America                           Artist      1949.0   
3                       Jamaica                           Artist      1945.0   
4      United States of America                       Researcher      1934.0   
...                         ...                              ...         ...   
23390                     Syria                       Politician      1984.0   
23391                 Australia                            Judge      1948.0   
23392  United States of America                          Athlete      1992.0   
23393        State of Palestine  Condottiero; resistance fighter      1977.0   
23394            United Kingdom                       Researcher      1946.0   

       Death year  Manner of death  Age of death  \
0          2001.0   natural causes          49.0   
1          2006.0   natural causes          91.0   
2          2002.0         accident          53.0   
3          1981.0   natural causes          36.0   
4          1996.0   natural causes          62.0   
...           ...              ...           ...   
23390      2019.0         homicide          35.0   
23391      2019.0   natural causes          71.0   
23392      2019.0         accident          27.0   
23393      2019.0  death in battle          42.0   
23394      2000.0   natural causes          54.0   

                                                   Words  
0      douglas noel adam march may wa english author ...  
1      augusto jos ramn pinochet ugarte pinoe also u ...  
2      linda lovelace born linda susan boreman januar...  
3      robert nesta marley om february may baptised b...  
4      carl edward sagan sen saygn november december ...  
...                                                  ...  
23390  hevrin khalaf kurdish hevrn xelef arabic also ...  
23391  richard ross sinclair tracey rfd qc august oct...  
23392  patrick day august october wa american profess...  
23393  baha abu alata arabic november november wa lea...  
23394                               span classmwheadline  

[23395 rows x 11 columns]

In [5]:
df['Gender'].value_counts()

Male                          19069
Female                         4304
Transgender Female               13
Transgender Male                  7
Intersex                          1
Transgender Female; Female        1
Name: Gender, dtype: int64

In [6]:
X = df[['Short description', 'Gender', 'Country', 'Occupation', 'Birth year']]
Y = df['Age of death']

In [7]:
# LabelEncoder

X_LE = np.zeros((len(X.T), len(X)))
LE = preprocessing.LabelEncoder()
for i in range(len(X.T)):
    LE.fit(X.T.to_numpy()[i])
    X_LE[i] = LE.transform(X.T.to_numpy()[i])
X_LE = X_LE.T

Y_np = Y.to_numpy()

In [8]:
# Standzardization

X_S = np.zeros((len(X_LE.T), len(X_LE)))
for i in range(len(X_LE.T)):
    X_S[i] = (X_LE.T[i]-np.min(X_LE.T[i]))/(np.max(X_LE.T[i])-np.min(X_LE.T[i]))
X_S = X_S.T

In [9]:
# Create training and test data sets

x = X_S
y = Y_np.astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = RANDOM_STATE)

## Grid Search with Cross Validation

In [ ]:
# list the parameters to be adjusted
param_grid = {"kernel": ['linear', 'poly', 'rbf'],
             "gamma": np.arange(1,21,1),
              "C": np.arange(1,21,1)}

# grid search with cross validation
grid_search = GridSearchCV(svm.SVR(), param_grid, cv = 5)
%time grid_search.fit(x_train,y_train)

# print the optimised parameter
print(grid_search.best_params_)

# print the rmse with the optimised parameter
print(mean_squared_error(y_test, grid_search.best_estimator_.predict(x_test), squared = False))